# Finding the Best Hyperparameters for BERTopic

In this notebook we will try to find the best hyperparameters for our BERTopic model, by trying different configurations of UMAP and HDBSCAN models. Then we will evaluate each model based on both standard evaluation metrics and manual inspection of the topics created. 

## 

## Imports & Setup

In [1]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

from sentence_transformers import SentenceTransformer

from utils.data_loader import DataLoader
from utils.bertopic_evaluator import BERTopicModelEvaluator
from config.model import NUM_TOPICS, TOP_K, EMBEDDING_MODEL, vectorizer_params, c_tfidf_params, metrics
from config.optimization import all_config_combinations, algos_dict

from bertopic.vectorizers import ClassTfidfTransformer

In [2]:
loader = DataLoader('../../data/data_speeches.csv', '../../data/data_statements.csv', split_data=False)
loader.process()

train_docs, train_sentences = loader.get_train_data()

For this step, we only need the training and validation data.

## Constant Model Initialization

We will use the same vectorizer, c-TF-IDF model and sentence transformer model for all our experiments. 

We're interesting in optimizing on the Dimensionality Reduction and Clustering models, so we will keep the rest of the pipeline constant.

In [3]:
vectorizer_model = CountVectorizer(**vectorizer_params)
ctfidf_model = ClassTfidfTransformer(**c_tfidf_params)
st_model = SentenceTransformer(EMBEDDING_MODEL)

Evaluating different UMAP and HDBSCAN configurations

## Hyperparameter Tuning

Let's see how many different configurations we can try:

In [4]:
search_space = all_config_combinations()
len(search_space)

260

Start from checkpointed model - paste last model that was being trained before the crash as a string. Example in the cell below.

In [5]:
start_from = "model_UMAP_{'n_components': 5, 'n_neighbors': 5, 'min_dist': 0.05}_HDBSCAN_{'min_cluster_size': 15, 'metric': 'euclidean', 'prediction_data': False}"

Do not touch, **uncomment only if you want to start from a checkpointed model**.

In [6]:
for i, config in enumerate(search_space):
    model_name = f"model_{config['dim_reduction_model']}_{config['dim_reduction_params']}_{config['clustering_model']}_{config['clustering_params']}"
    if model_name == start_from:
        start_from = i
        break
    
print(f"Starting from {start_from}")
search_space = search_space[start_from:]

Starting from 2


### Document Level

In [7]:
granularity = 'docs'

In [8]:
models = {}

print("Loading models...")

for config in search_space:
    dim_reduction_model = algos_dict[config['dim_reduction_model']](**config['dim_reduction_params'])
    clustering_model = algos_dict[config['clustering_model']](**config['clustering_params'])
        
    model_name = f"model_{config['dim_reduction_model']}_{config['dim_reduction_params']}_{config['clustering_model']}_{config['clustering_params']}"


    model = BERTopic(
        umap_model=dim_reduction_model,
        hdbscan_model=clustering_model,
        embedding_model=EMBEDDING_MODEL,
        vectorizer_model=vectorizer_model,
        ctfidf_model=ctfidf_model,
        nr_topics=NUM_TOPICS,
        top_n_words=TOP_K,
    )

    models[model_name] = model

Loading models...


In [9]:
evaluator = BERTopicModelEvaluator(
                                   models=models, 
                                   metrics=metrics, 
                                   dataset=train_docs,
                                   topics=NUM_TOPICS,
                                   top_k=TOP_K,
                                   granularity=granularity
                                   )                    

In [10]:
print("Evaluating models...")
evaluator.evaluate()

Evaluating models...
Training model:  model_UMAP_{'n_components': 5, 'n_neighbors': 5, 'min_dist': 0.05}_HDBSCAN_{'min_cluster_size': 15, 'metric': 'euclidean', 'prediction_data': False}
Model training complete.
Evaluating model:  model_UMAP_{'n_components': 5, 'n_neighbors': 5, 'min_dist': 0.05}_HDBSCAN_{'min_cluster_size': 15, 'metric': 'euclidean', 'prediction_data': False}
Evaluating metric coherence_c_npmi for model model_UMAP_{'n_components': 5, 'n_neighbors': 5, 'min_dist': 0.05}_HDBSCAN_{'min_cluster_size': 15, 'metric': 'euclidean', 'prediction_data': False}
Evaluating metric coherence_c_v for model model_UMAP_{'n_components': 5, 'n_neighbors': 5, 'min_dist': 0.05}_HDBSCAN_{'min_cluster_size': 15, 'metric': 'euclidean', 'prediction_data': False}
Evaluating metric coherence_u_mass for model model_UMAP_{'n_components': 5, 'n_neighbors': 5, 'min_dist': 0.05}_HDBSCAN_{'min_cluster_size': 15, 'metric': 'euclidean', 'prediction_data': False}
Evaluating metric coherence_c_uci for mod

/Users/dion/.pyenv/versions/3.11.8/envs/nlp-env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dion/.pyenv/versions/3.11.8/envs/nlp-env/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ZeroDivisionError: division by zero